# For this notebook please first download and extract the data and save the data in the directory specified in "data_location". Note that depending on how much data is in the location it might take very long and also need lots of RAM.

In [1]:
data_location = "selected_data"
save_to = "reduced_1_data"

In [2]:
#Imports
from os.path import join

import pandas as pd
import dask
import dask.dataframe as dd

import utils.preprocess

In [3]:
%%time
#save data here
utils.preprocess.initial_preprocess_data(  
    csv_path=join(data_location, "part-0000*.tsv"),
    output_dir = save_to,
    convert_tweet_id_to_number = False,
    seperate_bert = True,
    fully_drop_bert = True,
    drop_tweet_id = True,
    add_bert_length_as_feature = True,
    blocksize = '64MB')

The data is split into 242 partitions of size 64MB each.
Now computing unique media types... done (195.47719049453735 ms)
Now computing unique hashtag types... done (300.3290686607361 ms)
Now computing unique link types... done (326.5331690311432 ms)
Now computing unique domain types... done (355.00054454803467 ms)
Now executing main task... 

KeyboardInterrupt: 

In [4]:
test = pd.read_parquet(join(save_to,"nobert" ,"part-00018.parquet"))

In [5]:
test

,hashtags,medias,links,domains,type,language,timestamp,a_user_id,a_follower_count,a_following_count,...,b_follower_count,b_following_count,b_is_verified,b_account_creation,a_follows_b,reply,retweet,retweet_comment,like,bert_length
0,[0],[0],[1334254],[41635],TopLevel,E7F038DE3EAD397AEC9193686C911677,1614046081,5A1F20480F39102A83D01F9ED30896FE,11118,10135,...,98,139,False,1345639618,True,0,0,0,0,66
1,[0],[3],[0],[0],TopLevel,1F73BB863A39DB62B4A55B7E558DB1E8,1613487993,CD1A3CAC5A28824E9C7B86F75E712B24,305,555,...,689,183,False,1586978230,True,0,0,0,0,21
2,[0],"[2, 2, 2]",[0],[0],TopLevel,E7F038DE3EAD397AEC9193686C911677,1613611241,FCC08254BE5B51FBCA33CF08B03328FC,9141,107,...,68,729,False,1389770843,False,0,0,0,1613616076,25
3,[0],[0],[0],[0],TopLevel,E6936751CBF4F921F7DE1AEF33A16ED0,1613258425,A7E08E60BB7F94971FF1114D5177732F,9181,1325,...,75,364,False,1330442125,False,0,0,0,1613259936,16
4,[0],[2],[0],[0],Retweet,E7F038DE3EAD397AEC9193686C911677,1612601568,8508E0ADC05CAD60A6FB5F139F3E8D0A,162062,2480,...,377,1328,False,1488363116,False,0,0,0,1612602312,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129304,[0],"[2, 2, 2]",[0],[0],TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1612909905,210DD7ABE4B3E816891C2282035108BF,5689,3963,...,137,154,False,1275684050,True,1612914188,0,0,0,36
129305,[0],[0],[1347164],[31659],Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1612833473,2F81A2EB146FA8BF5C0A077204094616,1272,497,...,45,62,False,1229316407,True,0,0,0,0,39
129306,[0],[3],[0],[0],TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1613223070,0BC5A26080C741CABBE25BBBF2DCD5A7,12794,29,...,21,60,False,1589540358,False,0,0,0,1613226182,55
129307,[501497],[0],[0],[0],Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1613148996,191F517A154D5309BCB12DBCFA5A8858,263,463,...,1833,3641,False,1265081658,False,0,1613149814,0,0,54


In [22]:

test = dd.read_csv("selected_data\\part-00000.tsv",sep='\x01', header=None, blocksize="64MB", names=utils.preprocess.__all_columns)

In [23]:
for idx, i in enumerate(test.partitions):
    i = i.compute()
    i.to_parquet(join(save_to, f"part-{idx:05}.parquet"))

KeyboardInterrupt: 

In [24]:
test[0] = test[0].apply(lambda x: len(str(x).split("\t")))

In [25]:
test[0]

0           31
1           22
2           10
3           30
4           34
          ... 
3300765     79
3300766     84
3300767    195
3300768     44
3300769     60
Name: 0, Length: 3300770, dtype: int64